In [1]:
!wget https://huggingface.co/spaces/lmsys/mt-bench/resolve/main/data/mt_bench/model_judgment/gpt-4_single.jsonl
!wget https://huggingface.co/spaces/lmsys/mt-bench/resolve/main/data/mt_bench/model_judgment/gpt-4_pair.jsonl

--2024-09-18 04:51:26--  https://huggingface.co/spaces/lmsys/mt-bench/resolve/main/data/mt_bench/model_judgment/gpt-4_single.jsonl
Resolving huggingface.co (huggingface.co)... 99.86.102.109, 99.86.102.9, 99.86.102.115, ...
Connecting to huggingface.co (huggingface.co)|99.86.102.109|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/12/2b/122bd8e9eccbb3acc98acf73e0ecef3c96f24dcdb5f6639074ed304eb19f9cd4/76c55033c6b2b1cc3f62513458f84748a23352495fd42b1062a7401de5ff9bd9?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27gpt-4_single.jsonl%3B+filename%3D%22gpt-4_single.jsonl%22%3B&Expires=1726894286&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNjg5NDI4Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8xMi8yYi8xMjJiZDhlOWVjY2JiM2FjYzk4YWNmNzNlMGVjZWYzYzk2ZjI0ZGNkYjVmNjYzOTA3NGVkMzA0ZWIxOWY5Y2Q0Lzc2YzU1MDMzYzZiMmIxY2MzZjYyNTEzNDU4Zjg0NzQ4YTIzMzUyNDk1ZmQ0

99.86.74.13, 99.86.74.62, 99.86.74.116, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|99.86.74.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20113128 (19M) [text/plain]
Saving to: ‘gpt-4_single.jsonl’

gpt-4_single.jsonl  100%[===================>]  19.18M  41.0MB/s    in 0.5s    

2024-09-18 04:51:32 (41.0 MB/s) - ‘gpt-4_single.jsonl’ saved [20113128/20113128]

--2024-09-18 04:51:32--  https://huggingface.co/spaces/lmsys/mt-bench/resolve/main/data/mt_bench/model_judgment/gpt-4_pair.jsonl
Resolving huggingface.co (huggingface.co)... 99.86.102.9, 99.86.102.115, 99.86.102.109, ...
Connecting to huggingface.co (huggingface.co)|99.86.102.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/12/2b/122bd8e9eccbb3acc98acf73e0ecef3c96f24dcdb5f6639074ed304eb19f9cd4/d662c0b7d1d297f0494fcb4cc09fe8f054fa22d75deb4754a483a921984bc585?response-content-disposition=inline%3B+filename*%3DUT

In [1]:
!pip install -U plotly kaleido

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/e5/ae/580600f441f6fc05218bd6c9d5794f4aef072a7d9093b291f1c50a9db8bc/plotly-5.24.1-py3-none-any.whl.metadata
  Obtaining dependency information for kaleido from https://files.pythonhosted.org/packages/ae/b3/a0f0f4faac229b0011d8c4a7ee6da7c2dca0b6fd08039c95920846f23ca4/kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for tenacity>=6.2.0 from https://files.pythonhosted.org/packages/b6/cb/b86984bed139586d01532a587464b5805f12e397594f19f931c4c2fbfa61/tenacity-9.0.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 121.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 133.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [27]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


CATEGORIES = ["Writing", "Roleplay", "Reasoning", "Math", "Coding", "Extraction", "STEM", "Humanities"]


def get_model_df():
    cnt = 0
    q2result = []
    fin = open("/workspace/FastChat/fastchat/llm_judge/data/mt_bench/model_judgment/gpt-4_single.jsonl", "r")
    for line in fin:
        obj = json.loads(line)
        obj["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        q2result.append(obj)
    df = pd.DataFrame(q2result)
    return df

def toggle(res_str):
    if res_str == "win":
        return "loss"
    elif res_str == "loss":
        return "win"
    return "tie"

def get_model_df_pair():
    fin = open("gpt-4_pair.jsonl", "r")
    cnt = 0
    q2result = []
    for line in fin:
        obj = json.loads(line)

        result = {}
        result["qid"] = str(obj["question_id"])
        result["turn"] = str(obj["turn"])
        if obj["g1_winner"] == "model_1" and obj["g2_winner"] == "model_1":
            result["result"] = "win"
        elif obj["g1_winner"] == "model_2" and obj["g2_winner"] == "model_2":
            result["result"] = "loss"
        else:
            result["result"] = "tie"
        result["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        result["model"] = obj["model_1"]
        q2result.append(result)

    df = pd.DataFrame(q2result)

    return df

df = get_model_df()
df_pair = get_model_df_pair()


all_models = df["model"].unique()
print(all_models)
scores_all = []
for model in all_models:
    for cat in CATEGORIES:
        # filter category/model, and score format error (<1% case)
        res = df[(df["category"]==cat) & (df["model"]==model) & (df["score"] >= 0)]
        score = res["score"].mean()


        # scores_all.append({"model": model, "category": cat, "score": score, "winrate": winrate, "wtrate": winrate_adjusted})
        scores_all.append({"model": model, "category": cat, "score": score})
        
        
        
# target_models = ["Llama-2-7b-chat", "Llama-2-13b-chat", "Llama-2-70b-chat", "gpt-3.5-turbo", "claude-v1", "gpt-4"]

target_models = ['instr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round3',
 'nstr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round3',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round3',
 'instr_release_ultrafeedback_batch32_rejected_fully_from_zamba',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round2',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba',
 'gemma-2-2b-it', 'ultrafeedback_openhermespref_orcadoppairs_4epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs_then_1epoch',
 'HuggingFaceTB_SmolLM-1.7B-Instruct', 'microsoft_Phi-3-mini-4k-instruct',
 'Qwen_Qwen2-1.5B-Instruct', 'stabilityai_stablelm-2-1_6b-chat',
 'microsoft_Phi-3.5-mini-instruct', 'h2oai_h2o-danube3-4b-chat',
 'stabilityai_stablelm-zephyr-3b', 'h2oai_h2o-danube2-1.8b-chat',
 'mistralai_Mistral-7B-Instruct-v0.1']



scores_target = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models]

# sort by target_models
scores_target = sorted(scores_target, key=lambda x: target_models.index(x["model"]), reverse=True)

df_score = pd.DataFrame(scores_target)


# Drop specific models using DataFrame.drop()
models_to_drop = ['instr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round3',
 'nstr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round3',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round3',
 'instr_release_ultrafeedback_batch32_rejected_fully_from_zamba',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round2',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba',
 'ultrafeedback_openhermespref_orcadoppairs_4epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs_then_1epoch',
 'Qwen_Qwen2-1.5B-Instruct',
 'HuggingFaceTB_SmolLM-1.7B-Instruct', 'microsoft_Phi-3-mini-4k-instruct',
 'stabilityai_stablelm-2-1_6b-chat',
 'microsoft_Phi-3.5-mini-instruct', 'h2oai_h2o-danube3-4b-chat',
 'stabilityai_stablelm-zephyr-3b', 
 'h2oai_h2o-danube2-1.8b-chat']
#  'zamba2_1B_ultrafeedback_openhermespref_orcadoppairs_1epoch_then_1epoch_then_1epoch']

# Filter out the models to drop
df_score = df_score[~df_score['model'].isin(models_to_drop)]

rename_map = {'gemma-2-2b-it': 'Gemma2 2.6B instruct',
              'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs': "Zamba2 2.7B instruct",
              "h2oai_h2o-danube3-4b-chat": "h2o-danube3-4b-chat",
              "microsoft_Phi-3-mini-4k-instruct": "Phi-3-mini-4k-instruct",
              "microsoft_Phi-3.5-mini-instruct": "Phi-3.5-mini-instruct",
              'stabilityai_stablelm-zephyr-3b': 'stablelm-zephyr-3b',
              'mistralai_Mistral-7B-Instruct-v0.1': 'Mistral 7B Instruct-v0.1'
              }

for k, v in rename_map.items():
    df_score.replace(k, v, inplace=True)

# # Plot the updated DataFrame
# fig = px.line_polar(df_score, r='score', theta='category', line_close=True, category_orders={"category": CATEGORIES},
#                     color='model', markers=True, color_discrete_sequence=px.colors.qualitative.T10,)

fig = px.line_polar(df_score, r='score', theta='category', line_close=True, category_orders={"category": CATEGORIES},
                    color='model', markers=True, color_discrete_sequence=px.colors.qualitative.T10)

# Add an overall centered title to the plot and adjust the legend
fig.update_layout(
    title={
        'text': 'MT Bench',
        'x': 0.5,
        'y': 0.98,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    legend=dict(
        title_text=None,  # Remove the title of the legend
        orientation='h',  # Horizontal legend
        y=-0.2,  # Position the legend below the plot
        x=0.5,  # Center the legend horizontally
        xanchor='center',
        yanchor='top',
        itemwidth=50,  # Adjust the width of each legend item
        traceorder='normal'
    )
)


fig.show()

['instr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round3'
 'nstr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round2'
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat'
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round3'
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round3'
 'instr_release_ultrafeedback_batch32_rejected_fully_from_zamba'
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round2'
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round2'
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba'
 'gemma-2-2b-it' 'ultrafeedback_openhermespref_orcadoppairs_4epochs'
 'ultrafeedback_openhermespref_orcadoppairs_1epoch'
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs'
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs_then_1epoch'
 'HuggingFaceTB_SmolLM-1.7B-Instruct' 'microsoft

In [125]:
import json
import pandas as pd
import plotly.express as px


CATEGORIES = ["Writing", "Roleplay", "Reasoning", "Math", "Coding", "Extraction", "STEM", "Humanities"]


def get_model_df():
    cnt = 0
    q2result = []
    fin = open("/workspace/FastChat/fastchat/llm_judge/data/mt_bench/model_judgment/gpt-4_single.jsonl", "r")
    for line in fin:
        obj = json.loads(line)
        obj["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        q2result.append(obj)
    df = pd.DataFrame(q2result)
    return df

def toggle(res_str):
    if res_str == "win":
        return "loss"
    elif res_str == "loss":
        return "win"
    return "tie"

def get_model_df_pair():
    fin = open("gpt-4_pair.jsonl", "r")
    cnt = 0
    q2result = []
    for line in fin:
        obj = json.loads(line)

        result = {}
        result["qid"] = str(obj["question_id"])
        result["turn"] = str(obj["turn"])
        if obj["g1_winner"] == "model_1" and obj["g2_winner"] == "model_1":
            result["result"] = "win"
        elif obj["g1_winner"] == "model_2" and obj["g2_winner"] == "model_2":
            result["result"] = "loss"
        else:
            result["result"] = "tie"
        result["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        result["model"] = obj["model_1"]
        q2result.append(result)

    df = pd.DataFrame(q2result)

    return df

df = get_model_df()
df_pair = get_model_df_pair()


all_models = df["model"].unique()
scores_all = []
for model in all_models:
    for cat in CATEGORIES:
        res = df[(df["category"]==cat) & (df["model"]==model) & (df["score"] >= 0)]
        score = res["score"].mean()

        scores_all.append({"model": model, "category": cat, "score": score})


target_models = ['instr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round3',
 'nstr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round3',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round3',
 'instr_release_ultrafeedback_batch32_rejected_fully_from_zamba',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round2',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba',
 'ultrafeedback_openhermespref_orcadoppairs_4epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs_then_1epoch',
 'HuggingFaceTB_SmolLM-1.7B-Instruct', 'microsoft_Phi-3-mini-4k-instruct',
 'Qwen_Qwen2-1.5B-Instruct', 'stabilityai_stablelm-2-1_6b-chat',
 'microsoft_Phi-3.5-mini-instruct', 'h2oai_h2o-danube3-4b-chat',
 'stabilityai_stablelm-zephyr-3b', 'h2oai_h2o-danube2-1.8b-chat',
 'mistralai_Mistral-7B-Instruct-v0.1', 'gemma-2-2b-it']

scores_target = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models]

# sort by target_models
scores_target = sorted(scores_target, key=lambda x: target_models.index(x["model"]), reverse=True)

df_score = pd.DataFrame(scores_target)

# Drop specific models using DataFrame.drop()
models_to_drop = ['instr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round3',
 'nstr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round3',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round3',
 'instr_release_ultrafeedback_batch32_rejected_fully_from_zamba',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round2',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba',
 'ultrafeedback_openhermespref_orcadoppairs_4epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs_then_1epoch',
 'Qwen_Qwen2-1.5B-Instruct',
 'HuggingFaceTB_SmolLM-1.7B-Instruct', 'microsoft_Phi-3-mini-4k-instruct',
 'stabilityai_stablelm-2-1_6b-chat',
 'microsoft_Phi-3.5-mini-instruct', 'h2oai_h2o-danube3-4b-chat',
 'stabilityai_stablelm-zephyr-3b', 
 'h2oai_h2o-danube2-1.8b-chat']

# Filter out the models to drop
df_score = df_score[~df_score['model'].isin(models_to_drop)]

rename_map = {'gemma-2-2b-it': 'Gemma2 2.6B instruct',
              'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs': "Zamba2 2.7B instruct",
              "h2oai_h2o-danube3-4b-chat": "h2o-danube3-4b-chat",
              "microsoft_Phi-3-mini-4k-instruct": "Phi-3-mini-4k-instruct",
              "microsoft_Phi-3.5-mini-instruct": "Phi-3.5-mini-instruct",
              'stabilityai_stablelm-zephyr-3b': 'stablelm-zephyr-3b',
              'mistralai_Mistral-7B-Instruct-v0.1': 'Mistral 7B Instruct-v0.1'
              }

for k, v in rename_map.items():
    df_score.replace(k, v, inplace=True)

# Plot the updated DataFrame using a bar chart
fig = px.bar(df_score, x="category", y="score", color="model", barmode="group", 
             category_orders={"category": CATEGORIES}, 
             labels={"score": "", "category": ""},  # Set labels to empty strings to remove
             color_discrete_sequence=px.colors.qualitative.T10)

fig.update_layout(
    title={
        'text': "<b>MT Bench</b>",  # Add plot title
        'x': 0.5,  # Center the title
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=30, family="Arial",  # Specify a font family
                     color="black", weight="bold")
    },
    yaxis_title="Score",  # Add y-axis label
    xaxis_title="",  # You can leave the x-axis title empty or customize it
    xaxis=dict(
        tickfont=dict(size=18)  # Increase font size for x-axis labels
    ),
    yaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=20)
    ),
    legend=dict(
        title_text=None,  # Remove the title of the legend
        orientation='h',  # Horizontal legend
        y=-0.3,  # Position the legend below the plot
        x=0.5,  # Center the legend horizontally
        xanchor='center',
        yanchor='top',
        itemwidth=50,  # Adjust the width of each legend item
        traceorder='normal',
        font=dict(size=20)  # Increase font size for legend labels
    )
)

# Manually update the colors to emphasize Zamba2 and add borders to bars
for trace in fig.data:
    if trace.name == "Zamba2 2.7B instruct":
        trace.marker.color = "#FC8E86" 
        trace.opacity = 0.95
        trace.marker.line.width = 3
        trace.marker.line.color = "#AA5733"
        trace.name = "<b>Zamba2 2.7B instruct</b>"
    elif trace.name == "Mistral 7B Instruct-v0.1":
        trace.marker.color = "#B0E57C" 
        trace.opacity = 0.65
        trace.marker.line.width = 1
        trace.marker.line.color = 'black'
    elif trace.name == "Gemma2 2.6B instruct":
        trace.marker.color = "#A7C7E7" 
        trace.opacity = 0.65
        trace.marker.line.width = 1
        trace.marker.line.color = 'black'
    else:
        trace.opacity = 0.65 
    
    # Add borders to the bars
    
    

fig.show()


In [43]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


CATEGORIES = ["Writing", "Roleplay", "Reasoning", "Math", "Coding", "Extraction", "STEM", "Humanities"]


def get_model_df():
    cnt = 0
    q2result = []
    fin = open("/workspace/FastChat/fastchat/llm_judge/data/mt_bench/model_judgment/gpt-4_single.jsonl", "r")
    for line in fin:
        obj = json.loads(line)
        obj["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        q2result.append(obj)
    df = pd.DataFrame(q2result)
    return df

def toggle(res_str):
    if res_str == "win":
        return "loss"
    elif res_str == "loss":
        return "win"
    return "tie"

def get_model_df_pair():
    fin = open("gpt-4_pair.jsonl", "r")
    cnt = 0
    q2result = []
    for line in fin:
        obj = json.loads(line)

        result = {}
        result["qid"] = str(obj["question_id"])
        result["turn"] = str(obj["turn"])
        if obj["g1_winner"] == "model_1" and obj["g2_winner"] == "model_1":
            result["result"] = "win"
        elif obj["g1_winner"] == "model_2" and obj["g2_winner"] == "model_2":
            result["result"] = "loss"
        else:
            result["result"] = "tie"
        result["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        result["model"] = obj["model_1"]
        q2result.append(result)

    df = pd.DataFrame(q2result)

    return df

df = get_model_df()
df_pair = get_model_df_pair()


all_models = df["model"].unique()
print(all_models)
scores_all = []
for model in all_models:
    for cat in CATEGORIES:
        # filter category/model, and score format error (<1% case)
        res = df[(df["category"]==cat) & (df["model"]==model) & (df["score"] >= 0)]
        score = res["score"].mean()


        # scores_all.append({"model": model, "category": cat, "score": score, "winrate": winrate, "wtrate": winrate_adjusted})
        scores_all.append({"model": model, "category": cat, "score": score})
        
        
        
# target_models = ["Llama-2-7b-chat", "Llama-2-13b-chat", "Llama-2-70b-chat", "gpt-3.5-turbo", "claude-v1", "gpt-4"]

target_models = ['instr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round3',
 'nstr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round3',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round3',
 'instr_release_ultrafeedback_batch32_rejected_fully_from_zamba',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round2',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba',
 'gemma-2-2b-it', 'ultrafeedback_openhermespref_orcadoppairs_4epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs_then_1epoch',
 'HuggingFaceTB_SmolLM-1.7B-Instruct', 'microsoft_Phi-3-mini-4k-instruct',
 'Qwen_Qwen2-1.5B-Instruct', 'stabilityai_stablelm-2-1_6b-chat',
 'microsoft_Phi-3.5-mini-instruct', 'h2oai_h2o-danube3-4b-chat',
 'stabilityai_stablelm-zephyr-3b', 'h2oai_h2o-danube2-1.8b-chat',
 'zamba2_1B_ultrafeedback_openhermespref_orcadoppairs_1epoch_then_1epoch_then_1epoch']



scores_target = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models]

# sort by target_models
scores_target = sorted(scores_target, key=lambda x: target_models.index(x["model"]), reverse=True)

df_score = pd.DataFrame(scores_target)


# Drop specific models using DataFrame.drop()
models_to_drop = ['instr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round3',
 'nstr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round3',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round3',
 'instr_release_ultrafeedback_batch32_rejected_fully_from_zamba',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round2',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba',
 'ultrafeedback_openhermespref_orcadoppairs_4epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs_then_1epoch',
 'Qwen_Qwen2-1.5B-Instruct',
 # 'HuggingFaceTB_SmolLM-1.7B-Instruct', 
 'microsoft_Phi-3-mini-4k-instruct',
 # 'stabilityai_stablelm-2-1_6b-chat',
 'microsoft_Phi-3.5-mini-instruct', 'h2oai_h2o-danube3-4b-chat',
 'stabilityai_stablelm-zephyr-3b'
 # 'h2oai_h2o-danube2-1.8b-chat',
 # 'zamba2_1B_ultrafeedback_openhermespref_orcadoppairs_1epoch_then_1epoch_then_1epoch'
]

# Filter out the models to drop
df_score = df_score[~df_score['model'].isin(models_to_drop)]

rename_map = {'zamba2_1B_ultrafeedback_openhermespref_orcadoppairs_1epoch_then_1epoch_then_1epoch': 'Zamba2 1.2B instruct',
              'h2oai_h2o-danube2-1.8b-chat': "H2o-danube2 1.8B chat",
              'stabilityai_stablelm-2-1_6b-chat': "Stablelm2 1.6B chat",
              'HuggingFaceTB_SmolLM-1.7B-Instruct': 'SmolLM 1.7B instruct',
              'gemma-2-2b-it': 'Gemma2 2.6B instruct'}

for k, v in rename_map.items():
    df_score.replace(k, v, inplace=True)

# # Plot the updated DataFrame
# fig = px.line_polar(df_score, r='score', theta='category', line_close=True, category_orders={"category": CATEGORIES},
#                     color='model', markers=True, color_discrete_sequence=px.colors.qualitative.T10)

# # Add an overall centered title to the plot
# fig.update_layout(
#     title={
#         'text': 'MT Bench',
#         'x': 0.5,  # Center the title horizontally
#         'y': 0.95,
#         'xanchor': 'center',  # Align the title to the center
#         'yanchor': 'top'
#     }
# )


fig = px.line_polar(df_score, r='score', theta='category', line_close=True, category_orders={"category": CATEGORIES},
                    color='model', markers=True, color_discrete_sequence=px.colors.qualitative.T10)

# Add an overall centered title to the plot and adjust the legend
fig.update_layout(
    title={
        'text': 'MT Bench',
        'x': 0.5,
        'y': 0.98,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    legend=dict(
        title_text=None,  # Remove the title of the legend
        orientation='h',  # Horizontal legend
        y=-0.2,  # Position the legend below the plot
        x=0.5,  # Center the legend horizontally
        xanchor='center',
        yanchor='top',
        itemwidth=50,  # Adjust the width of each legend item
        traceorder='normal'
    )
)

fig.show()

['instr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round3'
 'nstr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round2'
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat'
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round3'
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round3'
 'instr_release_ultrafeedback_batch32_rejected_fully_from_zamba'
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round2'
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round2'
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba'
 'gemma-2-2b-it' 'ultrafeedback_openhermespref_orcadoppairs_4epochs'
 'ultrafeedback_openhermespref_orcadoppairs_1epoch'
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs'
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs_then_1epoch'
 'HuggingFaceTB_SmolLM-1.7B-Instruct' 'microsoft

In [124]:

import json
import pandas as pd
import plotly.express as px


CATEGORIES = ["Writing", "Roleplay", "Reasoning", "Math", "Coding", "Extraction", "STEM", "Humanities"]


def get_model_df():
    cnt = 0
    q2result = []
    fin = open("/workspace/FastChat/fastchat/llm_judge/data/mt_bench/model_judgment/gpt-4_single.jsonl", "r")
    for line in fin:
        obj = json.loads(line)
        obj["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        q2result.append(obj)
    df = pd.DataFrame(q2result)
    return df

def toggle(res_str):
    if res_str == "win":
        return "loss"
    elif res_str == "loss":
        return "win"
    return "tie"

def get_model_df_pair():
    fin = open("gpt-4_pair.jsonl", "r")
    cnt = 0
    q2result = []
    for line in fin:
        obj = json.loads(line)

        result = {}
        result["qid"] = str(obj["question_id"])
        result["turn"] = str(obj["turn"])
        if obj["g1_winner"] == "model_1" and obj["g2_winner"] == "model_1":
            result["result"] = "win"
        elif obj["g1_winner"] == "model_2" and obj["g2_winner"] == "model_2":
            result["result"] = "loss"
        else:
            result["result"] = "tie"
        result["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        result["model"] = obj["model_1"]
        q2result.append(result)

    df = pd.DataFrame(q2result)

    return df

df = get_model_df()
df_pair = get_model_df_pair()


all_models = df["model"].unique()
scores_all = []
for model in all_models:
    for cat in CATEGORIES:
        res = df[(df["category"]==cat) & (df["model"]==model) & (df["score"] >= 0)]
        score = res["score"].mean()

        scores_all.append({"model": model, "category": cat, "score": score})


target_models = ['zamba2_1B_ultrafeedback_openhermespref_orcadoppairs_1epoch_then_1epoch_then_1epoch',
 'instr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round3',
 'nstr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round3',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round3',
 'instr_release_ultrafeedback_batch32_rejected_fully_from_zamba',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round2',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba',
 'gemma-2-2b-it', 'ultrafeedback_openhermespref_orcadoppairs_4epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs_then_1epoch',
 'HuggingFaceTB_SmolLM-1.7B-Instruct', 'microsoft_Phi-3-mini-4k-instruct',
 'Qwen_Qwen2-1.5B-Instruct', 'stabilityai_stablelm-2-1_6b-chat',
 'microsoft_Phi-3.5-mini-instruct', 'h2oai_h2o-danube3-4b-chat',
 'stabilityai_stablelm-zephyr-3b', 'h2oai_h2o-danube2-1.8b-chat']

scores_target = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models]

# sort by target_models
scores_target = sorted(scores_target, key=lambda x: target_models.index(x["model"]), reverse=True)

df_score = pd.DataFrame(scores_target)

# Drop specific models using DataFrame.drop()
models_to_drop = ['instr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round3',
 'nstr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round3',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round3',
 'instr_release_ultrafeedback_batch32_rejected_fully_from_zamba',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round2',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba',
 'ultrafeedback_openhermespref_orcadoppairs_4epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs_then_1epoch',
 'Qwen_Qwen2-1.5B-Instruct',
 # 'HuggingFaceTB_SmolLM-1.7B-Instruct', 
 'microsoft_Phi-3-mini-4k-instruct',
 # 'stabilityai_stablelm-2-1_6b-chat',
 'microsoft_Phi-3.5-mini-instruct', 'h2oai_h2o-danube3-4b-chat',
 'stabilityai_stablelm-zephyr-3b'
 # 'h2oai_h2o-danube2-1.8b-chat',
 # 'zamba2_1B_ultrafeedback_openhermespref_orcadoppairs_1epoch_then_1epoch_then_1epoch'
]


# Filter out the models to drop
df_score = df_score[~df_score['model'].isin(models_to_drop)]

rename_map = {'zamba2_1B_ultrafeedback_openhermespref_orcadoppairs_1epoch_then_1epoch_then_1epoch': 'Zamba2 1.2B instruct',
              'h2oai_h2o-danube2-1.8b-chat': "H2o-danube2 1.8B chat",
              'stabilityai_stablelm-2-1_6b-chat': "Stablelm2 1.6B chat",
              'HuggingFaceTB_SmolLM-1.7B-Instruct': 'SmolLM 1.7B instruct',
              'gemma-2-2b-it': 'Gemma2 2.6B instruct'}

for k, v in rename_map.items():
    df_score.replace(k, v, inplace=True)

# Plot the updated DataFrame using a bar chart
fig = px.bar(df_score, x="category", y="score", color="model", barmode="group",
             category_orders={"category": CATEGORIES}, 
             labels={"score": "", "category": ""},  # Set labels to empty strings to remove
             color_discrete_sequence=px.colors.qualitative.T10)

fig.update_layout(
    title={
        'text': "<b>MT Bench</b>",  # Add plot title
        'x': 0.5,  # Center the title
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=30, family="Arial",  # Specify a font family
                     color="black", weight="bold")
    },
    yaxis_title="Score",  # Add y-axis label
    xaxis_title="",  # You can leave the x-axis title empty or customize it
    xaxis=dict(
        tickfont=dict(size=18)  # Increase font size for x-axis labels
    ),
    yaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=20)
    ),
    legend=dict(
        title_text=None,  # Remove the title of the legend
        orientation='h',  # Horizontal legend
        y=-0.3,  # Position the legend below the plot
        x=0.5,  # Center the legend horizontally
        xanchor='center',
        yanchor='top',
        itemwidth=50,  # Adjust the width of each legend item
        traceorder='normal',
        font=dict(size=20)  # Increase font size for legend labels
    )
)

# Manually update the colors to emphasize Zamba2 and add borders to bars
for trace in fig.data:
    trace.marker.line.width = 1
    trace.marker.line.color = 'black'
    if 'Zamba' in trace.name:
        trace.marker.color = "#FC8E86" 
        trace.opacity = 0.95
        trace.marker.line.width = 3
        trace.marker.line.color = "#AA5733"
        trace.name = "<b>Zamba2 1.2B instruct</b>"
    elif 'SmolLM' in trace.name:
        trace.marker.color = "#AADDAA" 
    # elif trace.name == "Mistral 7B Instruct-v0.1":
    #     trace.marker.color = "#B0E57C" 
    #     trace.opacity = 0.65
    # elif trace.name == "Gemma2 2.6B instruct":
    #     trace.marker.color = "#A7C7E7" 
    #     trace.opacity = 0.65
    else:
        trace.opacity = 0.65 
    
    # Add borders to the bars
    
    

fig.show()

In [ ]:
rename_map = {'zamba2_1B_ultrafeedback_openhermespref_orcadoppairs_1epoch_then_1epoch_then_1epoch': 'Zamba2-1.2B-instruct',
              'h2oai_h2o-danube2-1.8b-chat': "H2o-danube2-1.8b-chat",
              'stabilityai_stablelm-2-1_6b-chat': "Stablelm-2-1_6b-chat",
              'HuggingFaceTB_SmolLM-1.7B-Instruct': 'SmolLM-1.7B-Instruct',
              'gemma-2-2b-it': 'Gemma-2-2b-it'}

for k, v in rename_map.items():
    df_score.replace(k, v, inplace=True)

In [17]:
all_models = df["model"].unique()
print(all_models)
scores_all = []
for model in all_models:
    for cat in CATEGORIES:
        # filter category/model, and score format error (<1% case)
        res = df[(df["category"]==cat) & (df["model"]==model) & (df["score"] >= 0)]
        score = res["score"].mean()


        # scores_all.append({"model": model, "category": cat, "score": score, "winrate": winrate, "wtrate": winrate_adjusted})
        scores_all.append({"model": model, "category": cat, "score": score})

['instr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round3'
 'nstr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round2'
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat'
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round3'
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round3'
 'instr_release_ultrafeedback_batch32_rejected_fully_from_zamba'
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round2'
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round2'
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba'
 'gemma-2-2b-it' 'ultrafeedback_openhermespref_orcadoppairs_4epochs'
 'ultrafeedback_openhermespref_orcadoppairs_1epoch'
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs'
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs_then_1epoch'
 'HuggingFaceTB_SmolLM-1.7B-Instruct' 'microsoft

In [15]:
# target_models = ["Llama-2-7b-chat", "Llama-2-13b-chat", "Llama-2-70b-chat", "gpt-3.5-turbo", "claude-v1", "gpt-4"]

target_models = ['instr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round3',
 'nstr_release_ultrafeedback_batch32_rejected_fully_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round3',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round3',
 'instr_release_ultrafeedback_batch32_rejected_fully_from_zamba',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba_round2',
 'instr_release_ultrafeedback_infinfstruct_batch32_with_actual_mix_infinstr_ultrachat_round2',
 'instr_release_ultrafeedback_batch32_rejected_half_from_zamba',
 'gemma-2-2b-it', 'ultrafeedback_openhermespref_orcadoppairs_4epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs',
 'ultrafeedback_openhermespref_orcadoppairs_1epoch_then_2epochs_then_1epoch',
 'HuggingFaceTB_SmolLM-1.7B-Instruct', 'microsoft_Phi-3-mini-4k-instruct',
 'Qwen_Qwen2-1.5B-Instruct', 'stabilityai_stablelm-2-1_6b-chat',
 'microsoft_Phi-3.5-mini-instruct', 'h2oai_h2o-danube3-4b-chat',
 'stabilityai_stablelm-zephyr-3b', 'h2oai_h2o-danube2-1.8b-chat']



scores_target = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models]

# sort by target_models
scores_target = sorted(scores_target, key=lambda x: target_models.index(x["model"]), reverse=True)

df_score = pd.DataFrame(scores_target)
df_score = df_score[df_score["model"].isin(target_models)]

# rename_map = {"llama-13b": "LLaMA-13B",
#               "alpaca-13b": "Alpaca-13B",
#               "vicuna-33b-v1.3": "Vicuna-33B",
#               "vicuna-13b-v1.3": "Vicuna-13B",
#               "gpt-3.5-turbo": "GPT-3.5-turbo",
#               "claude-v1": "Claude-v1",
#               "gpt-4": "GPT-4"}

# for k, v in rename_map.items():
#     df_score.replace(k, v, inplace=True)

fig = px.line_polar(df_score, r = 'score', theta = 'category', line_close = True, category_orders = {"category": CATEGORIES},
                    color = 'model', markers=True, color_discrete_sequence=px.colors.qualitative.Pastel)

fig.show()

# scores_target = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models]

# # sort by target_models
# scores_target = sorted(scores_target, key=lambda x: target_models.index(x["model"]), reverse=True)

# df_score = pd.DataFrame(scores_target)
# df_score = df_score[df_score["model"].isin(target_models)]

# rename_map = {"llama-13b": "LLaMA-13B",
#               "alpaca-13b": "Alpaca-13B",
#               "vicuna-33b-v1.3": "Vicuna-33B",
#               "vicuna-13b-v1.3": "Vicuna-13B",
#               "gpt-3.5-turbo": "GPT-3.5-turbo",
#               "claude-v1": "Claude-v1",
#               "gpt-4": "GPT-4"}

# for k, v in rename_map.items():
#     df_score.replace(k, v, inplace=True)

# fig = px.line_polar(df_score, r = 'score', theta = 'category', line_close = True, category_orders = {"category": CATEGORIES},
#                     color = 'model', markers=True, color_discrete_sequence=px.colors.qualitative.Pastel)

# fig.show()

In [ ]:
# fig = px.line_polar(df_score, r = 'wtrate', theta = 'category', line_close = True, category_orders = {"category": CATEGORIES},
#                     color = 'model', markers=True, color_discrete_sequence=px.colors.qualitative.Pastel)
# fig.show()

In [ ]:
fig.update_layout(
    font=dict(
        size=18,
    ),
)
fig.write_image("fig.png", width=800, height=600, scale=2)